In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

is_available_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_available_cuda else 'cpu')

In [2]:
mnist_train = datasets.MNIST(
    root = '~/data',
    train = True,
    transform = ToTensor(), # need to review about torch.transform module
    download = True
)

mnist_test = datasets.MNIST(
    root = '~/data',
    train = False,
    transform = ToTensor(), # need to review about torch.transform module
    download = True
)

C:\Users\sungj\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
train_dataloader = DataLoader(mnist_train, batch_size=64, shuffle=True, drop_last = True)
test_dataloader = DataLoader(mnist_test, batch_size=64, shuffle=True, drop_last = True)

In [13]:
## dataloader 로 불러온 데이터의 차원
for X, Y in train_dataloader:
    X = X.view(-1, 28 * 28)
#     print(X.size()) #?# what is meaning of 2nd index = 1?
#     print(Y.size())
#     break

In [55]:
class MNIST_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.MLP = nn.Sequential(
            nn.Linear(784, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Softmax(dim = 0)
        )
        
    def forward(self, x):
        return self.MLP(x)

In [62]:
model = MNIST_MLP()

In [66]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)
criterion = nn.CrossEntropyLoss()

n_epoch = 100

dataloader = train_dataloader

for epoch in range(n_epoch):
    
    avg_cost = 0
    total_batch = len(dataloader)
    
    for X, Y in dataloader:
        X = X.view(-1, 28 * 28)
        y_predict = model(X)
        
        optimizer.zero_grad()
        cost = criterion(y_predict, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    if epoch % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 2.164844751
Epoch: 0011 cost = 2.164776087
Epoch: 0021 cost = 2.164572716
Epoch: 0031 cost = 2.164642334
Epoch: 0041 cost = 2.164785385
Epoch: 0051 cost = 2.164652824
Epoch: 0061 cost = 2.164707422
Epoch: 0071 cost = 2.164698124
Epoch: 0081 cost = 2.164623260
Epoch: 0091 cost = 2.164533377


생각보다 결과가 괜찮다

In [73]:
Y

tensor([1, 3, 7, 1, 3, 6, 6, 2, 2, 1, 0, 1, 3, 2, 8, 1, 2, 9, 4, 7, 5, 2, 8, 6,
        8, 6, 2, 8, 3, 3, 0, 2, 4, 6, 7, 4, 0, 3, 0, 5, 1, 0, 2, 2, 0, 7, 0, 8,
        2, 2, 7, 5, 9, 3, 2, 3, 6, 0, 8, 3, 5, 4, 7, 6])

In [72]:
torch.argmax(model(X), dim = 1)

tensor([1, 3, 7, 1, 3, 6, 6, 2, 2, 1, 0, 1, 3, 2, 8, 1, 2, 9, 4, 7, 5, 2, 8, 6,
        8, 6, 2, 8, 3, 3, 0, 2, 4, 6, 7, 9, 0, 3, 0, 5, 1, 0, 2, 2, 0, 7, 0, 8,
        2, 8, 7, 5, 9, 3, 2, 3, 6, 0, 8, 3, 5, 4, 7, 6])